In [ ]:
import pandas as pd
import  requests
import gmaps

from config import g_key

In [ ]:
city_data_df=pd.read_csv("./data/WeatherPy_Database.csv")
city_data_df

In [ ]:
city_data_df.dtypes

In [ ]:
gmaps.configure(api_key=gkey)

In [ ]:
min_temp = float(input("what is the minimum temperature you would like for your vacation"))
max_temp = float(input("whar is the maximum temperature you would like for your vacation"))

In [ ]:
rain_amount=input ("Do you want it to be raining (yes/no)")

snow_amount=input (" Do you want it to be snowing (yes/no)")

In [ ]:
if rain_amount =="no" and  snow_amount =="no":
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <=max_temp) &
                                          (city_data_df["Max Temp"]>= min_temp)&
                                          (city_data_df["Rain (inches)"]==0)&
                                          (city_data_df["Snow (inches)"]==0)]
elif rain_amount=="no" and snow_amount=="yes":
        filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <=max_temp) &
                                          (city_data_df["Max Temp"]>= min_temp)&
                                          (city_data_df["Rain (inches)"]==0)&
                                          (city_data_df["Snow (inches)"]>=0.0)]
elif rain_amount =="yes" and snow_amount=="no":
        filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <=max_temp) &
                                          (city_data_df["Max Temp"]>= min_temp)&
                                          (city_data_df["Rain (inches)"]>0.0)&
                                          (city_data_df["Snow (inches)"]==0)]
else:
        filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <=max_temp) &
                                          (city_data_df["Max Temp"]>= min_temp)&
                                          (city_data_df["Rain (inches)"]>0)&
                                          (city_data_df["Snow (inches)"]>0)]
    

In [ ]:
filtered_cities_df.head()

In [ ]:
filtered_cities_df.count()

In [ ]:
filtered_cities_df= filtered_cities_df.dropna()

In [ ]:
hotel_df = filtered_cities_df[["City","Country","Max Temp","Lat","Lng",]].copy()
hotel_df["Hotel Name"]==""
hotel_df.head()

In [ ]:
params ={
    "radius":5000,
    "type":"lodging",
    "key":g_key
}

for index, row in hotel_df.iterrows():
    lat=row["Lat"]
    lng=row["Lng"]
    
    params["location"]=f"{lat},{lng}"
    
    base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotels = requests.get(base_url, params=params).json()
    
    try:
        hotel_df.loc[index,"Hotel Name"]=hotels["results"][0]["name"]
    except IndexError:
        print("Hotel Not found...skipping...")

In [ ]:
hotel_df.head()

In [ ]:
output_data_file="./data/WeatherPy_Vacation.csv"
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [ ]:
vacation_df =pd.read_csv("./data/WeatherPy_Vacation.csv")
vacation_df.head()

In [ ]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [ ]:
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations=vacation_df[["Lat","Lng"]]

In [ ]:
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig=gmaps.figure()
fig.add_layer(marker_layer)

fig